### ENGR 418 Project Stage 1
Alif Aiman Ahmad Zukiman | 35338490 <br />
Anas Chellabi | 

In [1]:
# test

import numpy as np

a = [1,2,3]
b = [4,5,6]

c = np.array([a,b]).T
print(c)


[[1 4]
 [2 5]
 [3 6]]
